In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, BatchNormalization
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import features_engineering

from sklearn.model_selection import train_test_split

In [ ]:
X = pd.read_csv('/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT')


In [ ]:
# YEARS = [2019,2021,2022,2023,2024,2025]

# Data = filter_data_by_year_month(X, YEARS)

df = features_engineering(X)



In [ ]:
df

In [ ]:
# Assume `df` contains feature-engineered data, with 'target' as the label column
X = df.drop(columns=['target']).values  # Features
y = df['target'].values  # Target variable

# Reshape the data into sequences (timesteps)
timesteps = 10  # Number of timesteps for the RNN
X_sequences = []
y_sequences = []

for i in range(len(X) - timesteps):
    X_sequences.append(X[i:i+timesteps])
    y_sequences.append(y[i+timesteps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)


In [ ]:
X_train[0][0]

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X1, y))
# # Calculate sizes for train and validation sets
# dataset_size = len(X1)  # Total number of samples in your dataset
# train_size = int(0.8 * dataset_size)

# # Shuffle the dataset if needed
# dataset = dataset.shuffle(buffer_size=dataset_size)

# # Split the dataset
# train_dataset = dataset.take(train_size)       # Take the first 80%
# val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# # Batch both datasets if needed
# batch_size = 32
# train_dataset = train_dataset.batch(batch_size)
# val_dataset = val_dataset.batch(batch_size)

In [ ]:
tf.random.set_seed(1234) # for consistent results
# Define the model
model = Sequential([
    # First GRU Layer
    GRU(128, input_shape=(10, X_sequences.shape[2]), return_sequences=True, name="GRU_Layer1"),
    Dropout(0.3, name="Dropout_Layer1"),  # Regularization
    BatchNormalization(name="BatchNorm_Layer1"),  # Normalize to stabilize training
    
    # Second GRU Layer
    GRU(64, return_sequences=False, name="GRU_Layer2"),
    Dropout(0.3, name="Dropout_Layer2"),

    # Fully Connected Dense Layers
    Dense(128, activation='relu', name="Dense_Layer1"),
    Dropout(0.3, name="Dropout_Layer3"),  # Regularization
    Dense(64, activation='relu', name="Dense_Layer2"),

    # Output Layer for Binary Classification
    Dense(1, activation='sigmoid', name="Output_Layer")  # Predicting probability of class 1
])

In [ ]:
model.summary()

In [ ]:
model = tf.keras.models.load_model("../models/")

In [ ]:

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  # Binary classification loss
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),      # Adam optimizer
    metrics=['accuracy']                                         # Track accuracy
)

# Learning rate reduction callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,          # Reduce learning rate by a factor of 0.5
    patience=3,          # Wait 3 epochs with no improvement before reducing
    min_lr=1e-6          # Set a minimum learning rate
)

# Model checkpoint callback
checkpoint = ModelCheckpoint(
    filepath='../models/RNN_model_2.keras',  # Save the model to this path
    monitor='val_accuracy',                 # Monitor validation accuracy
    mode='max',                             # Save when accuracy is maximized
    save_best_only=True,                    # Save only if the model improves
    verbose=1                               # Display a message when saving
)

# Train the model
history = model.fit(
    X_train, y_train,                         # Training data
    validation_data=(X_test, y_test),         # Validation data
    epochs=20,                                # Train for 20 epochs
    batch_size=32,                            # Batch size
    callbacks=[reduce_lr, checkpoint]         # Use the callbacks
)
